# Overview of Prompt Engineering Techniques & Best Practices

## Part 1: Prompt Engineering Best Practices

In this section, we provide an overview of the top tips and best practices for prompting LLMs.

---

We first load the necessary libraries:

In [2]:
import warnings
import os

import openai
import gradio as gr

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv, find_dotenv
import IPython

In [3]:
_ = load_dotenv(find_dotenv()) # read local .env file
MODEL = 'meta-llama/Llama-2-70b-chat-hf'
warnings.filterwarnings('ignore')
openai.api_base = os.getenv("ANYSCALE_API_BASE", os.getenv("OPENAI_API_BASE"))
openai.api_key = os.getenv("ANYSCALE_API_KEY", os.getenv("OPENAI_API_KEY"))

In [4]:
# completion function
def get_completion(messages, model=MODEL, temperature=1, max_tokens=300):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

### Be Specific and Clear

Write instructions as clear and specific as possible to get the desired LLM behaviors:

In [5]:
global_trending_movies = ["The Suicide Squad", "Bend it like Beckman", "Pele", "No Time to Die", "Dune",  "Spider-Man: No Way Home", "The French Dispatch", "Black Widow", "Eternals", "The Matrix Resurrections", "West Side Story", "The Many Saints of Newark"]

system_message = """
Your task is to recommend movies to a customer.

You are responsible to recommend a movie from the top global trending movies from {global_trending_movies}.

You should refrain from asking users for their preferences and avoid asking for personal information.

If you don't have a movie to recommend or don't know the user interests, you should respond "Sorry, couldn't find a movie to recommend today".
"""

user_request = """
Please recommend a movie based on my interests.
"""

message = [
    {
        "role": "system",
        "content": system_message.format(global_trending_movies=global_trending_movies)
    },
    {
        "role": "user",
        "content": user_request
    }
]

response = get_completion(message)
print(response)

  Sure, I can recommend a movie based on your interests! From the top global trending movies, I would recommend "Spider-Man: No Way Home". It's a superhero movie that has been gaining a lot of attention and has received positive reviews. It's an action-packed movie that promises to deliver a thrilling experience. If you're interested in watching a movie with a mix of action, adventure, and fantasy, then this might be the right choice for you.


The more specific the desired the behavior you want from the model, the more specific the instructions and logic should be. Below is an example where the customer provides information about interests:

In [6]:
global_trending_movies = ["The Suicide Squad", "Pele", "Bend it like Beckham", "No Time to Die", "Dune",  "Spider-Man: No Way Home", "The French Dispatch", "Black Widow", "Eternals", "The Matrix Resurrections", "West Side Story", "The Many Saints of Newark"]

system_message = """
Your task is to recommends movies to a customer.

You are responsible to recommend a movie from the top global trending movies from {global_trending_movies}.

You should refrain from asking users for their preferences and avoid asking for personal information.

Provide a short descripition of each movie if more than one. If you don't have a movie to recommend or don't know the user interests, you should respond "Sorry, couldn't find a movie to recommend today.".
"""

user_request = """
I love soccer. Please recommend a movie based on my interests.
"""

message = [
    {
        "role": "system",
        "content": system_message.format(global_trending_movies=global_trending_movies)
    },
    {
        "role": "user",
        "content": user_request
    }
]

response = get_completion(message)
print(response)

  Great! Based on your interest in soccer, I would recommend "Bend it like Beckham"! 🏈🏆

"Bend it like Beckham" is a heartwarming and uplifting movie about a young Indian girl who dreams of becoming a professional soccer player, despite her traditional family's disapproval. The movie follows her journey as she learns to balance her cultural heritage with her passion for the sport. With stunning visuals, catchy music, and inspiring performances, this movie is a must-watch for any soccer fan. Plus, it's a great choice for the whole family! 🌟

If you're interested in watching a movie that'll leave you feeling motivated and energized, "Bend it like Beckham" is the perfect choice for you! 💪🏼🎬

I hope you enjoy the movie! 😊


### Add Delimiters

Adding delimiters help to better structure instructions and the overall prompt components. This is beneficial to get more reliable responses.

In [7]:
prompt = """
Convert the following code block in the #### <code> #### section to Python:

####
strings2.push("one")
strings2.push("two")
strings2.push("THREE")
strings2.push("4")
####
"""

message = [
    {
        "role": "user",
        "content": prompt
    }
]

IPython.display.Markdown("```python" + get_completion(message) + "\n```")

```python  Sure! Here's the equivalent code in Python:
```
strings2 = []
strings2.append("one")
strings2.append("two")
strings2.append("THREE")
strings2.append("4")
```
In Python, we use `append` instead of `push` to add elements to a list. Also, note that we don't need to explicitly declare the data type of `strings2` since Python is dynamically typed and can infer the data type from the context.
```

### Specify Output Format

If the format of prompt responses are important, then this should be explicitly stated in the prompt to get desired results. In the example, we would like to export the results as a JSON object.

In [8]:
prompt = """
Your task is: given a product description, return the requested information in the section delimited by ### ###. Format the output as a JSON object.

Product Description: Introducing the Nike Air Max 270 React: a comfortable and stylish sneaker that combines two of Nike's best technologies. With a sleek black design and a unique bubble sole, these shoes are perfect for everyday wear.

###
product_name: the name of the product
product_bran: the name of the brand (if any)
###
"""

message = [
    {
        "role": "user",
        "content": prompt
    }
]

print(get_completion(message))

  {
"product_name": "Nike Air Max 270 React",
"product_brand": "Nike"
}


### Think Step by Step

To elicit reasoning in LLMs, you can prompt the model to think step-by-step. Prompting the model in this way allows it to provide the details steps before providing a final response that solves the problem.

In [9]:
prompt = """The odd numbers in this group may add up to an even or odd number: 15, 32, 5, 13, 82, 7, 1.

Solve by breaking the problem into steps. First, identify the odd numbers, add them, and indicate whether the result is odd or even."""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response= get_completion(messages, temperature=1)

print(response)

  Sure! Let's break down the problem into steps:

Step 1: Identify the odd numbers in the group.

The odd numbers in the group are 5, 13, 7, and 1.

Step 2: Add the odd numbers together.

5 + 13 + 7 + 1 = 26

Step 3: Determine whether the result is odd or even.

26 is an even number.

Therefore, the sum of the odd numbers in the group is 26, which is an even number.


In [10]:
prompt = """From these numbers in this group identify all prime numbers: 15, 32, 5, 13, 82, 7, 1.

Solve by breaking the problem into steps. First, identify the prime numbers, add them, and indicate whether the result is odd or even."""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response= get_completion(messages)

print(response)

  Sure! Here's how we can identify the prime numbers in the given group and determine whether their sum is odd or even:

Step 1: Identify the prime numbers in the group:

* 5 is prime.
* 13 is prime.
* 7 is prime.

Step 2: Add the prime numbers:

5 + 13 + 7 = 25

Step 3: Determine whether the sum is odd or even:

25 is odd.

Therefore, the prime numbers in the group are 5, 13, and 7, and their sum is odd.


### Role Playing

The example below shows how to apply role playing using a chat model like GPT-3.5 Turbo. Notice the use of system message, user message, and assistant message in the example. You can combine different messages to mimic or jump start the behavior you want or expect from the model.

In [24]:
system_message = """
The following is a conversation with an AI research assistant. The assistant tone is technical and scientific.
"""

user_message_1 = """
Hello, who are you?
"""

ai_message_1 = """
Greeting! I am an AI research assistant. How can I help you today?
"""

prompt = """
Human: Can you tell me the number of stars in our Milky Way Galaxy?
AI:
"""

messages = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": user_message_1
    },
    {
        "role": "assistant",
        "content": ai_message_1

    },
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(messages)
print(response)

  Sure! The number of stars in the Milky Way Galaxy is estimated to be between 200 and 400 billion. The exact number is difficult to determine because many stars are too distant or too faint to be detected using current telescope technology. However, ongoing surveys and missions are continually providing new data that are helping scientists refine their estimates. For example, the European Space Agency's Gaia mission, which launched in 2013, has been mapping the stars in the Milky Way Galaxy with unprecedented precision and has released several datasets that have been used to estimate the number of stars in the galaxy. The most recent estimate, based on the Gaia Data Release 2 (DR2) catalog, suggests that there are approximately 292 billion stars in the Milky Way. However, this number is likely to continue to be refined as new data becomes available.
